# **Agricultural drought in central South America - trend analysis**

In [3]:
source("../methods_extRemes.r")

# list to convert between RCM and GCM (to find the correct GSAT file)
gsats = list('CCCma-CanESM2' = "CanESM2", 'CNRM-CERFACS-CNRM-CM5' = "CNRM-CM5", 'CSIRO-QCCCE-CSIRO-Mk3-6-0' = "CSIRO-Mk3-6-0", 'ICHEC-EC-EARTH' = "EC-EARTH",
             'IPSL-IPSL-CM5A-MR' = "IPSL-CM5A-LR", 'MIROC-MIROC5' = "MIROC5", 'MPI-M-MPI-ESM-LR' = "MPI-ESM-LR", "MOHC-HadGEM2-ES" = "HadGEM2-ES",
             "NCC-NorESM1-M" = "NorESM1-M", "NOAA-GFDL-GFDL-ESM2M" = "GFDL-ESM2G", "MPI-M-MPI-ESM-MR" = "MPI-ESM-MR")

# generic method to compute bootstrapped confidence intervals
boot_ci <- function(mdl, rp, cov1, cov2, ev = NA, nsamp = 10000, lower = F, rel = F) {
    
    n <- length(mdl$x)
    mtype <- strsplit(mdl$type, "_")[[1]]
    
    fun <- get(paste0("f",mtype[1]))
        
    boot_sample <- rbind.fill(sapply(1:nsamp, function(i) {
        
        # resample & refit model
        boot_df <- mdl$cov.data[sample(1:n,n,replace = T),]
        suppressWarnings(boot_fit <- fun(mdl$var.name, mdl$cov.name, boot_df, type = mtype[2], method = "MLE"))
        
        # compute required quantities for bootstrap (dispersion, PR, delta_I, delta_logI)
        pars_now <- sgev_pars(boot_fit, cov1)
        disp <- pars_now$scale / pars_now$loc
        sigma <- pars_now$scale
        trend <- boot_fit$results$par["alpha"]
        
        if(is.na(ev)) {
            rl <- return_level(boot_fit, rp, cov1, lower = lower)
            rlhist <- return_level(boot_fit, rp, cov2, lower = lower)
        } else {
            rl <- ev
            rp <- return_period(boot_fit, ev, cov1, lower = lower)
            rlhist <- return_level(boot_fit, rp, cov2, lower = lower)
        }
        
        pr <- prob_ratio(boot_fit, rl, cov1, cov2, lower = lower)
        dI <- delta_I(boot_fit, rp, cov1, cov2, rel = rel, lower = lower)
        
        if(mtype[1] == "lnorm") {
            dlogI <- (log(rl) - log(rlhist)) / log(rlhist) * 100
            return(as.data.frame(list("disp" = disp, "sigma" = sigma, "trend" = trend, "rp" = rp, "pr" = pr, "delta_I" = dI, "delta_logI" = dlogI)))
        } else {
            return(as.data.frame(list("disp" = disp, "sigma" = sigma, "trend" = trend, "rp" = rp, "pr" = pr, "delta_I" = dI)))
        }
    }, simplify = F))
    return(apply(boot_sample, 2, quantile, c(0.025, 0.975)))
}

---
## **CORDEX**

### **Precip**

In [ ]:
rp <- 20
nboot = 1000
lower = T
dI_rel = T

cordex_pr <- rbind.fill(sapply(list.files("ts", pattern = "pr-ond_SAM"), function(fnm) {
    
    set.seed(1)
    
    # print(fnm)
    if(grepl("r12i1p1", fnm)) {
        gmst_fnm <- '/home/clair/wwa/00_gsat/cordex_tas_Amon_EC-EARTH_rcp85_r12i1p1.dat'
    } else {
        gcm <- strsplit(fnm, "_")[[1]][4]
        gmst_fnm <- list.files("/home/clair/wwa/00_gsat", pattern = gsats[[gcm]], full.names = T)
    }
        
    if(length(gmst_fnm) == 0) {return(NULL)}
    
    # load data + GCM GMST into a single dataframe
    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", "precip"))
    gmst <- read.fwf(gmst_fnm, comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    event_gmst <- df["2022", "gmst"]
    
    mdl_eval <- flnorm("precip", "gmst", df[df$year >= 1980 & df$year <= 2022,], type = "fixeddisp", method = "MLE")  # obs period only
    mdl_attr <- flnorm("precip", "gmst", df[df$year <= 2022,], type = "fixeddisp", method = "MLE")                    # up to present day
    mdl_proj <- flnorm("precip", "gmst", df[df$year <= 2050,], type = "fixeddisp", method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc                                # needs bootstrapping
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_dlogI <- (log(attr_rl) - log(attr_rlhist)) / log(attr_rlhist) * 100
    attr_trend <- mdl_attr$results$par["alpha"]
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_dlogI <- (log(proj_rl) - log(proj_rlfut)) / log(proj_rlfut) * 100
    proj_trend <- mdl_proj$results$par["alpha"]
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_logvalue" = log10(attr_rl), "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "attr_dlogI" = attr_dlogI, "attr_dlogI_lower" = ci_attr["2.5%","delta_logI"], "attr_dlogI_upper" = ci_attr["97.5%","delta_logI"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"],
                             "proj_dlogI" = proj_dlogI, "proj_dlogI_lower" = ci_proj["2.5%","delta_logI"], "proj_dlogI_upper" = ci_proj["97.5%","delta_logI"]
                             )))
    
}, simplify = F))
write.csv(cordex_pr, "cordex-pr-results.csv", row.names = F)

---
### **Water balance**
Fit normal distribution: data is not very non-normal  
Can't use fixed-dispersion distribution because data can be both positive and negative: use shift fit instead.

In [16]:
rp <- 20
nboot = 1000
lower = T
dI_rel = T

cordex_cwb_ond <- rbind.fill(sapply(list.files("ts", pattern = "cwb-ond_SAM"), function(fnm) {
    
    set.seed(1)
    
    if(grepl("r12i1p1", fnm)) {
        gmst_fnm <- '/home/clair/wwa/00_gsat/cordex_tas_Amon_EC-EARTH_rcp85_r12i1p1.dat'
    } else {
        gcm <- strsplit(fnm, "_")[[1]][4]
        gmst_fnm <- list.files("/home/clair/wwa/00_gsat", pattern = gsats[[gcm]], full.names = T)
    }
        
    if(length(gmst_fnm) == 0) {return(NULL)}
    
    # load data + GCM GMST into a single dataframe
    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", "precip"))
    gmst <- read.fwf(gmst_fnm, comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    event_gmst <- df["2022", "gmst"]
    
    mdl_eval <- fnorm("precip", "gmst", df[df$year >= 1980 & df$year <= 2022,], type = "shift", method = "MLE")  # obs period only
    mdl_attr <- fnorm("precip", "gmst", df[df$year <= 2022,], type = "shift", method = "MLE")                    # up to present day
    mdl_proj <- fnorm("precip", "gmst", df[df$year <= 2050,], type = "shift", method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc
    eval_sigma <- eval_pars_now$scale
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = T)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = T)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = T)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = T)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = T)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = T, lower = T)   
    attr_trend <- mdl_attr$results$par["alpha"]
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = T)
    proj_rp <- return_period(mdl_proj, attr_rl, event_gmst + 0.8, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = T)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = T)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = T)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = T, lower = T)
    proj_trend <- mdl_proj$results$par["alpha"]

    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- (boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = T, rel = dI_rel))
    ci_attr <- (boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = T, rel = dI_rel))
    ci_proj <- (boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = T, rel = dI_rel))
    
    # return results
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "eval_sigma" = eval_sigma, "eval_sigma_lower" = ci_eval["2.5%","sigma"], "eval_sigma_upper" = ci_eval["97.5%","sigma"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"]
                             )))
    
}, simplify = F))
write.csv(cordex_cwb_ond, "cordex-cwb-results.csv", row.names = F)

---
### **tas**

In [7]:
# method to obtain 95% confidence interval by bootstrapping
boot_ci_tas <- function(mdl, rp, cov1, cov2, ev = NA, nsamp = 10000, lower = F, rel = F) {
    
    n <- length(mdl$x)
    
    boot_sample <- rbind.fill(sapply(1:nsamp, function(i) {
        
        # resample & refit model
        boot_df <- mdl$cov.data[sample(1:n,n,replace = T),]
        suppressWarnings(boot_fit <- fnorm("tas", "gmst", boot_df, type = "shift", method = "MLE"))
        
        # compute required quantities for bootstrap (dispersion, PR, delta_I, delta_logI)
        pars_now <- sgev_pars(boot_fit, cov1)
        sigma <- pars_now$scale
        trend <- boot_fit$results$par["alpha"]
        
        rl <- return_level(boot_fit, rp, cov1, lower = lower)
        rlhist <- return_level(boot_fit, rp, cov2, lower = lower)
        
        if(is.na(ev)) {
            pr <- prob_ratio(boot_fit, rl, cov1, cov2, lower = lower)
        } else {
            pr <- prob_ratio(boot_fit, ev, cov1, cov2, lower = lower)
        }
        
        dI <- delta_I(boot_fit, rp, cov1, cov2, rel = rel, lower = lower)   
        
        as.data.frame(list("sigma" = sigma, "trend" = trend, "pr" = pr, "delta_I" = dI))
        
    }, simplify = F))
    return(apply(boot_sample, 2, quantile, c(0.025, 0.975)))
}

In [61]:
rp <- 2
nboot = 1000
lower = F
dI_rel = F

cordex_tas_ond <- suppressWarnings(rbind.fill(sapply(list.files("ts", pattern = "tas-ond_SAM")[1], function(fnm) {
    
    set.seed(1)
    
    if(grepl("r12i1p1", fnm)) {
        gmst_fnm <- '/home/clair/wwa/00_gsat/cordex_tas_Amon_EC-EARTH_rcp85_r12i1p1.dat'
    } else {
        gcm <- strsplit(fnm, "_")[[1]][4]
        gmst_fnm <- list.files("/home/clair/wwa/00_gsat", pattern = gsats[[gcm]], full.names = T)
    }
        
    if(length(gmst_fnm) == 0) {return(NULL)}
    
    # load data + GCM GMST into a single dataframe
    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", "tas"))
    gmst <- read.fwf(gmst_fnm, comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    event_gmst <- df["2022", "gmst"]
    
    mdl_eval <- fnorm("tas", "gmst", df[df$year >= 1980 & df$year <= 2022,], type = "shift", method = "MLE")  # obs period only
    mdl_attr <- fnorm("tas", "gmst", df[df$year <= 2022,], type = "shift", method = "MLE")                    # up to present day
    mdl_proj <- fnorm("tas", "gmst", df[df$year <= 2050,], type = "shift", method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_sigma <- eval_pars_now$scale
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_sigma <- attr_pars_now$scale   
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_trend <- mdl_attr$results$par["alpha"]
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_sigma <- proj_pars_now$scale    
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_trend <- mdl_proj$results$par["alpha"]

    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, ev = attr_rl, cov1 = event_gmst, cov2 = event_gmst + 0.8, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_sigma" = eval_sigma, "eval_sigma_lower" = ci_eval["2.5%","sigma"], "eval_sigma_upper" = ci_eval["97.5%","sigma"],
                             "attr_sigma" = attr_sigma, "attr_sigma_lower" = ci_attr["2.5%","sigma"], "attr_sigma_upper" = ci_attr["97.5%","sigma"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"]
                             )))
    
}, simplify = F)))
# write.csv(cordex_tas_ond, "cordex-tas-ond-results.csv", row.names = F)

ERROR: Error in boot_ci_tas(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - : could not find function "boot_ci_tas"


---
### **mrso**

In [13]:
rp <- 20
nboot = 100
lower = T
dI_rel = T

cordex_mrso <- suppressWarnings(rbind.fill(sapply(list.files("ts", pattern = "mrso"), function(fnm) {
    
    set.seed(1)
    
    # print(fnm)
    if(grepl("ERA5", fnm)) {
        gmst <- read.fwf("/home/clair/wwa/90_data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
    } else if(grepl("r12i1p1", fnm)) {
        gmst_fnm <- '/home/clair/wwa/00_gsat/cordex_tas_Amon_EC-EARTH_rcp85_r12i1p1.dat'
        gmst <- read.fwf(gmst_fnm, comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
    } else {
        gcm <- strsplit(fnm, "_")[[1]][4]
        gmst_fnm <- list.files("/home/clair/wwa/00_gsat", pattern = gsats[[gcm]], full.names = T)
        if(length(gmst_fnm) == 0) {return(NULL)}
        gmst <- read.fwf(gmst_fnm, comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
    }

    # load data + GCM GMST into a single dataframe
    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", "mrso"))
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    event_gmst <- df["2022", "gmst"]
    event_value <- df["2022", "mrso"]
    
    mdl_eval <- fnorm("mrso", "gmst", df[df$year >= 1980 & df$year <= 2022,], type = "shift", method = "MLE")  # obs period only
    mdl_attr <- fnorm("mrso", "gmst", df[df$year <= 2022,], type = "shift", method = "MLE")                    # up to present day
    mdl_proj <- fnorm("mrso", "gmst", df[df$year <= 2050,], type = "shift", method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc                                # needs bootstrapping
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    eval_rp <- return_period(mdl_eval, event_value, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    # attr_dlogI <- (log(attr_rl) - log(attr_rlhist)) / log(attr_rlhist) * 100
    attr_trend <- mdl_attr$results$par["alpha"]
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    # proj_dlogI <- (log(proj_rl) - log(proj_rlfut)) / log(proj_rlfut) * 100
    proj_trend <- mdl_proj$results$par["alpha"]
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, ev = event_value, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "eval_rp" = eval_rp, "eval_rp_lower" = ci_eval["2.5%","rp"], "eval_rp_upper" = ci_eval["97.5%","rp"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_logvalue" = log10(attr_rl), "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             # "attr_dlogI" = attr_dlogI, "attr_dlogI_lower" = ci_attr["2.5%","delta_logI"], "attr_dlogI_upper" = ci_attr["97.5%","delta_logI"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"]
                             # "proj_dlogI" = proj_dlogI, "proj_dlogI_lower" = ci_proj["2.5%","delta_logI"], "proj_dlogI_upper" = ci_proj["97.5%","delta_logI"]
                             )))
    
}, simplify = F)))
write.csv(cordex_mrso, "cordex-mrso-results.csv", row.names = F)

## **CMIP6**

### **Precip**

In [13]:
rp <- 20
nboot = 10
lower = T
dI_rel = T

cmip6_pr <- suppressWarnings(rbind.fill(sapply(list.files("ts-raw", pattern = "pr\\."), function(fnm) {

    set.seed(1)
    
    df <- read.fwf(paste0("ts-raw/",fnm), widths = c(6,10), comment.char = "#", col.names = c("year", "precip"))
    
    gmst_fnm <- gsub("pr", "SGSAT2022", fnm)
    gmst <- read.csv(paste0("ts-raw/",gmst_fnm), comment.char = "#", sep = " ", col.names = c("year", "gmst"))
    
    df <- merge(gmst, df)
    row.names(df) <- df$year
    event_gmst <- df["2022", "gmst"]
    
    mdl_eval <- flnorm("precip", "gmst", df[df$year >= 1980 & df$year <= 2022,], type = "fixeddisp", method = "MLE")  # obs period only
    mdl_attr <- flnorm("precip", "gmst", df[df$year <= 2022,], type = "fixeddisp", method = "MLE")                    # up to present day
    mdl_proj <- flnorm("precip", "gmst", df[df$year <= 2050,], type = "fixeddisp", method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc                                # needs bootstrapping
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_dlogI <- (log(attr_rl) - log(attr_rlhist)) / log(attr_rlhist) * 100
    attr_trend <- mdl_attr$results$par["alpha"]
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_dlogI <- (log(proj_rl) - log(proj_rlfut)) / log(proj_rlfut) * 100
    proj_trend <- mdl_proj$results$par["alpha"]
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_logvalue" = log10(attr_rl), "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "attr_dlogI" = attr_dlogI, "attr_dlogI_lower" = ci_attr["2.5%","delta_logI"], "attr_dlogI_upper" = ci_attr["97.5%","delta_logI"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"],
                             "proj_dlogI" = proj_dlogI, "proj_dlogI_lower" = ci_proj["2.5%","delta_logI"], "proj_dlogI_upper" = ci_proj["97.5%","delta_logI"]
                             )))
}, simplify = F)))
write.csv(cmip6_pr, "cmip6-pr-results.csv", row.names = F)

[1] "pr.ACCESS-CM2_r1i1p1f1.hssp585.dat"
[1] "pr.ACCESS-ESM1-5_r1i1p1f1.hssp585.dat"
[1] "pr.CAMS-CSM1-0_r1i1p1f1.hssp585.dat"
[1] "pr.CanESM5_r1i1p1f1.hssp585.dat"
[1] "pr.CMCC-ESM2_r1i1p1f1.hssp585.dat"
[1] "pr.CNRM-CM6-1_r1i1p1f2.hssp585.dat"
[1] "pr.CNRM-CM6-1-HR_r1i1p1f2.hssp585.dat"
[1] "pr.CNRM-ESM2-1_r1i1p1f2.hssp585.dat"
[1] "pr.EC-Earth3_r1i1p1f1.hssp585.dat"
[1] "pr.EC-Earth3-CC_r1i1p1f1.hssp585.dat"
[1] "pr.EC-Earth3-Veg_r1i1p1f1.hssp585.dat"
[1] "pr.EC-Earth3-Veg-LR_r1i1p1f1.hssp585.dat"
[1] "pr.FGOALS-g3_r1i1p1f1.hssp585.dat"
[1] "pr.GFDL-ESM4_r1i1p1f1.hssp585.dat"
[1] "pr.HadGEM3-GC31-LL_r1i1p1f3.hssp585.dat"
[1] "pr.HadGEM3-GC31-MM_r1i1p1f3.hssp585.dat"
[1] "pr.INM-CM4-8_r1i1p1f1.hssp585.dat"
[1] "pr.INM-CM5-0_r1i1p1f1.hssp585.dat"
[1] "pr.IPSL-CM6A-LR_r1i1p1f1.hssp585.dat"
[1] "pr.KACE-1-0-G_r1i1p1f1.hssp585.dat"
[1] "pr.MIROC-ES2L_r1i1p1f2.hssp585.dat"
[1] "pr.MIROC6_r1i1p1f1.hssp585.dat"
[1] "pr.MPI-ESM1-2-HR_r1i1p1f1.hssp585.dat"
[1] "pr.MPI-ESM1-2-LR_r1i1p1f1.hssp5

### **Temp**

In [10]:
rp <- 2
nboot = 10
lower = F
dI_rel = F

fit <- fnorm
fittype <- "shift"
varnm <- "tas"

cmip6_pr <- suppressWarnings(rbind.fill(sapply(list.files("ts-raw", pattern = "tas"), function(fnm) {
    
    set.seed(1)
    
    print(fnm)
    df <- read.fwf(paste0("ts-raw/",fnm), widths = c(6,10), comment.char = "#", col.names = c("year", varnm))
    
    gmst_fnm <- gsub(varnm, "SGSAT2022", fnm)
    gmst <- read.csv(paste0("ts-raw/",gmst_fnm), comment.char = "#", sep = " ", col.names = c("year", "gmst"))
    
    df <- merge(gmst, df)
    row.names(df) <- df$year
    event_gmst <- df["2022", "gmst"]
    
    mdl_eval <- fit(varnm, "gmst", df[df$year >= 1980 & df$year <= 2022,], type = fittype, method = "MLE")  # obs period only
    mdl_attr <- fit(varnm, "gmst", df[df$year <= 2022,], type = fittype, method = "MLE")                    # up to present day
    mdl_proj <- fit(varnm, "gmst", df[df$year <= 2050,], type = fittype, method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc
    eval_sigma <- eval_pars_now$scale
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_trend <- mdl_attr$results$par["alpha"]
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_trend <- mdl_proj$results$par["alpha"]
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "eval_sigma" = eval_sigma, "eval_sigma_lower" = ci_eval["2.5%","sigma"], "eval_sigma_upper" = ci_eval["97.5%","sigma"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_logvalue" = log10(attr_rl), "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"]
                             )))
}, simplify = F)))
write.csv(cmip6_pr, "cmip6-tas-results.csv", row.names = F)

[1] "tas.ACCESS-CM2_r1i1p1f1.hssp585.dat"
[1] "tas.ACCESS-ESM1-5_r1i1p1f1.hssp585.dat"
[1] "tas.CAMS-CSM1-0_r1i1p1f1.hssp585.dat"
[1] "tas.CanESM5_r1i1p1f1.hssp585.dat"
[1] "tas.CMCC-ESM2_r1i1p1f1.hssp585.dat"
[1] "tas.CNRM-CM6-1_r1i1p1f2.hssp585.dat"
[1] "tas.CNRM-CM6-1-HR_r1i1p1f2.hssp585.dat"
[1] "tas.CNRM-ESM2-1_r1i1p1f2.hssp585.dat"
[1] "tas.EC-Earth3_r1i1p1f1.hssp585.dat"
[1] "tas.EC-Earth3-CC_r1i1p1f1.hssp585.dat"
[1] "tas.EC-Earth3-Veg_r1i1p1f1.hssp585.dat"
[1] "tas.EC-Earth3-Veg-LR_r1i1p1f1.hssp585.dat"
[1] "tas.FGOALS-g3_r1i1p1f1.hssp585.dat"
[1] "tas.GFDL-ESM4_r1i1p1f1.hssp585.dat"
[1] "tas.HadGEM3-GC31-LL_r1i1p1f3.hssp585.dat"
[1] "tas.HadGEM3-GC31-MM_r1i1p1f3.hssp585.dat"
[1] "tas.INM-CM4-8_r1i1p1f1.hssp585.dat"
[1] "tas.INM-CM5-0_r1i1p1f1.hssp585.dat"
[1] "tas.IPSL-CM6A-LR_r1i1p1f1.hssp585.dat"
[1] "tas.KACE-1-0-G_r1i1p1f1.hssp585.dat"
[1] "tas.MIROC-ES2L_r1i1p1f2.hssp585.dat"
[1] "tas.MIROC6_r1i1p1f1.hssp585.dat"
[1] "tas.MPI-ESM1-2-HR_r1i1p1f1.hssp585.dat"
[1] "tas.MPI-

### **Effective precip (pr - PET)**

In [ ]:
rp <- 20
nboot = 10
lower = T
dI_rel = T

fit <- fnorm
fittype <- "shift"
varnm <- "preff"

cmip6_pr <- suppressWarnings(rbind.fill(sapply(list.files("ts-raw", pattern = varnm), function(fnm) {

    set.seed(1)
    print(fnm)
    df <- read.fwf(paste0("ts-raw/",fnm), widths = c(5,10), comment.char = "#", col.names = c("year", varnm))
    
    gmst_fnm <- gsub(varnm, "SGSAT2022", fnm)
    gmst <- read.csv(paste0("ts-raw/",gmst_fnm), comment.char = "#", sep = " ", col.names = c("year", "gmst"))
    
    df <- merge(gmst, df)
    row.names(df) <- df$year
    event_gmst <- df["2022", "gmst"]
    
    mdl_eval <- fit(varnm, "gmst", df[df$year >= 1980 & df$year <= 2022,], type = fittype, method = "MLE")  # obs period only
    mdl_attr <- fit(varnm, "gmst", df[df$year <= 2022,], type = fittype, method = "MLE")                    # up to present day
    mdl_proj <- fit(varnm, "gmst", df[df$year <= 2050,], type = fittype, method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc                                # needs bootstrapping
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_trend <- mdl_attr$results$par["alpha"]
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_trend <- mdl_proj$results$par["alpha"]
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_logvalue" = log10(attr_rl), "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"]
                             )))
}, simplify = F)))
write.csv(cmip6_pr, "cmip6-preff-results.csv", row.names = F)

---
## **UKCP18**

### **Precip**

In [ ]:
rp <- 20
nboot = 1000
lower = T
dI_rel = T

ukcp_pr <- rbind.fill(sapply(list.files("ts", pattern = "pr-ond_ukcp"), function(fnm) {
    
    set.seed(1)
    
    gmst_fnm <- paste0("/home/clair/wwa/00_gsat/ukcp-land-gcm_gsat-aw_sm_",strsplit(fnm, "_")[[1]][4])
        
    # load data + GCM GMST into a single dataframe
    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", "precip"))
    gmst <- read.csv(gmst_fnm, comment.char = "#", header = F, sep = " ", col.names = c("year", "gmst"))
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    event_gmst <- df["2022", "gmst"]
    
    mdl_eval <- flnorm("precip", "gmst", df[df$year >= 1980 & df$year <= 2022,], type = "fixeddisp", method = "MLE")  # obs period only
    mdl_attr <- flnorm("precip", "gmst", df[df$year <= 2022,], type = "fixeddisp", method = "MLE")                    # up to present day
    mdl_proj <- flnorm("precip", "gmst", df[df$year <= 2050,], type = "fixeddisp", method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc                                # needs bootstrapping
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_dlogI <- (log(attr_rl) - log(attr_rlhist)) / log(attr_rlhist) * 100
    attr_trend <- mdl_attr$results$par["alpha"]
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_dlogI <- (log(proj_rl) - log(proj_rlfut)) / log(proj_rlfut) * 100
    proj_trend <- mdl_proj$results$par["alpha"]
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_logvalue" = log10(attr_rl), "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "attr_dlogI" = attr_dlogI, "attr_dlogI_lower" = ci_attr["2.5%","delta_logI"], "attr_dlogI_upper" = ci_attr["97.5%","delta_logI"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"],
                             "proj_dlogI" = proj_dlogI, "proj_dlogI_lower" = ci_proj["2.5%","delta_logI"], "proj_dlogI_upper" = ci_proj["97.5%","delta_logI"]
                             )))
    
}, simplify = F))
write.csv(ukcp_pr, "ukcp-pr-results.csv", row.names = F)

### **Temp**

In [42]:
rp <- 2
nboot = 1000
lower = F
dI_rel = F

ukcp_tas <- suppressWarnings(rbind.fill(sapply(list.files("ts", pattern = "tas-ond_ukcp"), function(fnm) {
    
    set.seed(1)
    
    gmst_fnm <- paste0("/home/clair/wwa/00_gsat/ukcp-land-gcm_gsat-aw_sm_",strsplit(fnm, "_")[[1]][4])
            
    # load data + GCM GMST into a single dataframe
    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", "tas"))
    gmst <- read.csv(gmst_fnm, comment.char = "#", header = F, sep = " ", col.names = c("year", "gmst"))
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    event_gmst <- df["2022", "gmst"]
    
    mdl_eval <- fnorm("tas", "gmst", df[df$year >= 1980 & df$year <= 2022,], type = "shift", method = "MLE")  # obs period only
    mdl_attr <- fnorm("tas", "gmst", df[df$year <= 2022,], type = "shift", method = "MLE")                    # up to present day
    mdl_proj <- fnorm("tas", "gmst", df[df$year <= 2050,], type = "shift", method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_sigma <- eval_pars_now$scale
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_sigma <- attr_pars_now$scale   
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_trend <- mdl_attr$results$par["alpha"]
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_sigma <- proj_pars_now$scale    
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_trend <- mdl_proj$results$par["alpha"]

    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, ev = attr_rl, cov1 = event_gmst, cov2 = event_gmst + 0.8, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_sigma" = eval_sigma, "eval_sigma_lower" = ci_eval["2.5%","sigma"], "eval_sigma_upper" = ci_eval["97.5%","sigma"],
                             "attr_sigma" = attr_sigma, "attr_sigma_lower" = ci_attr["2.5%","sigma"], "attr_sigma_upper" = ci_attr["97.5%","sigma"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"]
                             )))
    
}, simplify = F)))
write.csv(ukcp_tas, "ukcp-tas-results.csv", row.names = F)

--- 
## **Station data**

In [ ]:
source("../methods_extRemes.r")

### **Precip**

In [ ]:
rp <- 20
nboot = 1000
lower = T
dI_rel = T

gmst <- read.fwf("/home/clair/wwa/90_data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
event_gmst <- df["2022", "gmst"]

stn_pr <- rbind.fill(sapply(list.files("ts", pattern = "pr-ond_station"), function(fnm) {
    # fnm <- list.files("ts", pattern = "pr-ond_station")[6]

    set.seed(1)
            
    # load data + GCM GMST into a single dataframe
    df <- merge(gmst, read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", "precip")))
    row.names(df) <- df$year

    mdl <- flnorm("precip", "gmst", df[!is.na(df$precip),], type = "fixeddisp", method = "MLE")
    
    pars_now <- sgev_pars(mdl, event_gmst)
    disp <- pars_now$scale / pars_now$loc
    sigma <- pars_now$scale
    
    ev <- df["2022","precip"]
    
    rp <- return_period(mdl, ev, event_gmst, lower = lower)
    pr <- prob_ratio(mdl, ev, event_gmst, event_gmst - 1.2, lower = lower)
    dI <- delta_I(mdl, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)
    
    mdl_ci <- boot_ci(mdl, rp = rp, ev = ev, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("disp" = disp, "disp_lower" = mdl_ci["2.5%","disp"], "disp_upper" = mdl_ci["97.5%","disp"],
                             "sigma" = sigma, "sigma_lower" = mdl_ci["2.5%","sigma"], "sigma_upper" = mdl_ci["97.5%","sigma"],
                             "trend" = unname(mdl$results$par["alpha"]), "trend_lower" = mdl_ci["2.5%","trend"], "trend_upper" = mdl_ci["97.5%","trend"],
                             "event_value" = ev, "logvalue" = log10(ev), 
                             "rp" = rp, "rp_lower" = mdl_ci["2.5%","rp"], "rp_upper" = mdl_ci["2.5%","rp"],
                             "pr" = pr, "pr_lower" = mdl_ci["2.5%","pr"], "attr_pr_upper" = mdl_ci["97.5%","pr"],
                             "dI" = dI, "dI_lower" = mdl_ci["2.5%","delta_I"], "attr_dI_upper" = mdl_ci["97.5%","delta_I"]
                             )))
    
}, simplify = F))

write.csv(stn_pr, "station-pr-results.csv", row.names = F)

## **HighResMIP**

### **Precip**

In [83]:
rp <- 20
nboot = 1000
lower = T
dI_rel = T

fit <- flnorm
fittype <- "fixeddisp"
varnm <- "pr"

gmst <- read.fwf("/home/clair/wwa/90_data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
event_gmst <- gmst[gmst$year == "2022", "gmst"]

res_pr <- suppressWarnings(rbind.fill(sapply(list.files("ts", pattern = paste0("_",varnm, "-ond_highresSST")), function(fnm) {
    
    set.seed(1)
        
    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", varnm))
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    mdl_eval <- fit(varnm, "gmst", df[df$year >= 1980 & df$year <= 2022,], type = fittype, method = "MLE")  # obs period only
    mdl_attr <- fit(varnm, "gmst", df[df$year <= 2022,], type = fittype, method = "MLE")                    # up to present day
    mdl_proj <- fit(varnm, "gmst", df[df$year <= 2050,], type = fittype, method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc
    eval_sigma <- eval_pars_now$scale
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_trend <- unname(mdl_attr$results$par["alpha"])
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_trend <- unname(mdl_proj$results$par["alpha"])
    
    # log params
    attr_dlogI <- (log10(attr_rl) - log10(attr_rlhist)) / log10(attr_rlhist) * 100
    proj_dlogI <- (log10(proj_rl) - log10(proj_rlfut)) / log10(proj_rlfut) * 100
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "eval_sigma" = eval_sigma, "eval_sigma_lower" = ci_eval["2.5%","sigma"], "eval_sigma_upper" = ci_eval["97.5%","sigma"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_logvalue" = log10(attr_rl), "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "attr_dlogI" = attr_dlogI, "attr_dlogI_lower" = ci_attr["2.5%","delta_logI"], "attr_dlogI_upper" = ci_attr["97.5%","delta_logI"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"],
                             "proj_dlogI" = proj_dlogI, "proj_dlogI_lower" = ci_proj["2.5%","delta_logI"], "proj_dlogI_upper" = ci_proj["97.5%","delta_logI"]
                             )))
}, simplify = F)))
write.csv(res_pr, paste0("highresmip-",varnm,"-results.csv"), row.names = F)

### **Temperature**

In [80]:
rp <- 2
nboot = 1000
lower = F
dI_rel = F

fit <- fnorm
fittype <- "shift"
varnm <- "tas"

gmst <- read.fwf("/home/clair/wwa/90_data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
event_gmst <- gmst[gmst$year == "2022", "gmst"]

res <- suppressWarnings(rbind.fill(sapply(list.files("ts", pattern = paste0("_",varnm, "-ond_highresSST")), function(fnm) {
    
    set.seed(1)

    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", varnm))
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    mdl_eval <- fit(varnm, "gmst", df[df$year >= 1980 & df$year <= 2022,], type = fittype, method = "MLE")  # obs period only
    mdl_attr <- fit(varnm, "gmst", df[df$year <= 2022,], type = fittype, method = "MLE")                    # up to present day
    mdl_proj <- fit(varnm, "gmst", df[df$year <= 2050,], type = fittype, method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc
    eval_sigma <- eval_pars_now$scale
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_trend <- unname(mdl_attr$results$par["alpha"])
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_trend <- unname(mdl_proj$results$par["alpha"])
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "eval_sigma" = eval_sigma, "eval_sigma_lower" = ci_eval["2.5%","sigma"], "eval_sigma_upper" = ci_eval["97.5%","sigma"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"]
                             )))
}, simplify = F)))
write.csv(res, paste0("highresmip-",varnm,"-results.csv"), row.names = F)

[1] "CSA-drought_tas-ond_highresSST_CMCC-CM2-HR4.dat"
[1] "CSA-drought_tas-ond_highresSST_CMCC-CM2-VHR4.dat"
[1] "CSA-drought_tas-ond_highresSST_CNRM-CM6-1-HR.dat"
[1] "CSA-drought_tas-ond_highresSST_CNRM-CM6-1.dat"
[1] "CSA-drought_tas-ond_highresSST_EC-Earth3P-HR.dat"
[1] "CSA-drought_tas-ond_highresSST_EC-Earth3P.dat"
[1] "CSA-drought_tas-ond_highresSST_HadGEM3-GC31-HM.dat"
[1] "CSA-drought_tas-ond_highresSST_HadGEM3-GC31-LM.dat"
[1] "CSA-drought_tas-ond_highresSST_HadGEM3-GC31-MM.dat"
[1] "CSA-drought_tas-ond_highresSST_MPI-ESM1-2-HR.dat"
[1] "CSA-drought_tas-ond_highresSST_MPI-ESM1-2-XR.dat"


### **Effective precip**

In [81]:
rp <- 20
nboot = 1000
lower = T
dI_rel = T

fit <- fnorm
fittype <- "shift"
varnm <- "effpr"

gmst <- read.fwf("/home/clair/wwa/90_data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
event_gmst <- gmst[gmst$year == "2022", "gmst"]

res_pr <- suppressWarnings(rbind.fill(sapply(list.files("ts", pattern = paste0(varnm, "-ond_highresSST")), function(fnm) {
    
    set.seed(1)
    
    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", varnm))
    
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    mdl_eval <- fit(varnm, "gmst", df[df$year >= 1980 & df$year <= 2022,], type = fittype, method = "MLE")  # obs period only
    mdl_attr <- fit(varnm, "gmst", df[df$year <= 2022,], type = fittype, method = "MLE")                    # up to present day
    mdl_proj <- fit(varnm, "gmst", df[df$year <= 2050,], type = fittype, method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc
    eval_sigma <- eval_pars_now$scale
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_trend <- unname(mdl_attr$results$par["alpha"])
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_trend <- unname(mdl_proj$results$par["alpha"])
    
    # log params
    attr_dlogI <- (log10(attr_rl) - log10(attr_rlhist)) / log10(attr_rlhist) * 100
    proj_dlogI <- (log10(proj_rl) - log10(proj_rlfut)) / log10(proj_rlfut) * 100
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "eval_sigma" = eval_sigma, "eval_sigma_lower" = ci_eval["2.5%","sigma"], "eval_sigma_upper" = ci_eval["97.5%","sigma"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"]
                             )))
}, simplify = F)))
write.csv(res_pr, paste0("highresmip-",varnm,"-results.csv"), row.names = F)